# Visualing Spatial Data in Python: An Overview 

#### Contact: Rossano Schifanella, schifane@di.unito.it

## Mapping in Python with geopandas

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import geopandas as gpd
import pysal as ps

import palettable as pltt
import seaborn as sns
from seaborn import palplot
import numpy as np




## Loading up spatial data 

The easiest way to get from a file to a quick visualization of the data is by loading it as a `GeoDataFrame` and calling the `plot` command. The main library employed for all of this is `geopandas` which is a geospatial extension of the `pandas` library, already introduced before. `geopandas` supports exactly the same functionality that `pandas` does (in fact since it is built on top of it, so most of the underlying machinery is pure `pandas`), plus a wide range of spatial counterparts that make manipulation and general "munging" of spatial data as easy as non-spatial tables.

In two lines of code, we will obtain a graphical representation of the spatial data contained in a file that can be in many formats; actually, since it uses the same drivers under the hood, you can load pretty much the same kind of vector files that QGIS permits. Let us start by plotting single layers in a crude but quick form, and we will build style and sophistication into our plots later on.

* Polygons

Let us begin with the most common type of spatial data in the social science: polygons. For example, we can load the geography of MSOAs in London with the following lines of code:

In [ ]:
msoas_london_link = 'data/shp/MSOA_London.shp'
msoas_london = gpd.read_file(msoas_london_link)
msoas_london.head()

In [ ]:
msoas_london.plot()

This might not be the most aesthetically pleasant visual representation of the MSOAs geography, but it is hard to argue it is not quick to produce. We will work on styling and customizing spatial plots later on.

If you call a single row of the `geometry` column, it'll return a small plot ith the shape:

In [ ]:
msoas_london.loc[10, 'geometry']

* Lines

Displaying lines is as straight-forward as polygons. To load tube lines in London:

In [ ]:
london_tube_link = 'data/shp/tublines.shp'
london_tube = gpd.read_file(london_tube_link)

In [ ]:
london_tube.plot()

* Points

Finally, points follow a similar structure. If we want to represent named places in London:

In [ ]:
london_pois_link = 'data/shp/osm_pois_london.shp'
london_pois = gpd.read_file(london_pois_link)

In [ ]:
london_pois.plot()

In [ ]:
london_pois = london_pois.to_crs({'datum': 'OSGB36',
    'k': 0.999601272,
    'lat_0': 49,
    'lon_0': -2,
    'no_defs': True,
    'proj': 'tmerc',
    'units': 'm',
    'x_0': 400000,
    'y_0': -100000
})

In [ ]:
london_pois['fclass'].unique()

In [ ]:
london_pubs = london_pois.loc[london_pois['fclass']=="pub"]
london_pubs.info()

In [ ]:
london_pubs.plot()

## Styling plots

It is possible to tweak several aspects of a plot to customize if to particular needs. In this section, we will explore some of the basic elements that will allow us to obtain more compelling maps.

* Remove axis
* Set title
* Customizing colors, lines, alpha
* Customizing size

These are styling options. Omit them or alter them to suit.

In [ ]:
# from matplotlib import rc

# rc('font', **{'family':'DejaVu Sans',
#     'sans-serif':['Helvetica'],
#     'monospace': ['Inconsolata'],
#     'serif': ['Adobe Garamond Pro']}
#   )

In [ ]:
# Setup figure and axis
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')

# Plot layer of polygons on the axis
msoas_london.plot(ax=ax, linewidth=0.1, facecolor='#999999', edgecolor='#111111', alpha=0.8)
# Display
plt.title('Greater London MSOAs')
plt.tight_layout()
plt.show()    

## Composing multi-layer maps

So far we have considered many aspects of plotting a single layer of data. However, in many cases, an effective map will require more than one: for example we might want to display streets on top of the polygons of neighborhoods, and add a few points for specific locations we want to highlight. At the very heart of GIS is the possibility to combine spatial information from different sources by overlaying it on top of each other, and this is fully supported in Python.

Essentially, combining different layers on a single map boils down to adding each of them to the same axis in a sequential way, as if we were literally overlaying one on top of the previous one. For example, let us get the simplest possible plot, one with the polygons from the MSOAs and the tube lines on top of them:

In [ ]:

# Setup figure and axis
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')

# Plot layer of polygons on the axis
msoas_london.plot(ax=ax, linewidth=0.05, facecolor='#999999', edgecolor='#111111', alpha=0.8)
london_tube.plot(ax=ax, linewidth=0.5, edgecolor='red', alpha=0.6)
london_pubs.plot(ax=ax, linewidth=0.1, facecolor='blue', alpha=0.8, markersize=1)
# Display
plt.title('Greater London MSOAs+tube+pubs')
plt.tight_layout()
plt.show()    

## Using palettes to create aesthetically pleasant maps

The choice of colors can influence the look and, ultimately, the effectiveness of a map. Although in some cases picking colors that simply allow you to distinguish the different elements might suffice, sometimes, you want to convey certain feelings (warmth, safety, etc.). In those cases, using preexisting palettes can be useful.

In this section, we will learn how to use pre-existent palettes to style your maps. We will be using the library [`palettable`](https://jiffyclub.github.io/palettable/), which provides many "canned" palettes. We will also use the handy function `palplot` (from the library `seaborn`) to examine the colors quickly.

For the sake of the example, let us use a palette based on one of Wes Anderson's movies, Darjeeling Limited:

<img src='http://66.media.tumblr.com/2815b755b493555dd4a74fc9f7c84bdb/tumblr_nj7cclt9qb1tvvqeko1_500.jpg'></img>

Here is how you can pull out those colors:

In [ ]:
wes = pltt.wesanderson.Darjeeling2_5.hex_colors
palplot(wes)

Now, note how the object `wes` simply contains a list of colors in the hex standard:

In [ ]:
wes

In [ ]:
# Setup figure and axis
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')
ax.set_title('Greater London MSOAs+tube+pubs')

# Plot layer of polygons on the axis
msoas_london.plot(ax=ax, linewidth=0.05, facecolor='#D5E3D8', edgecolor='#D5E3D8', alpha=0.8)
london_tube.plot(ax=ax, linewidth=0.5, edgecolor='#618A98', alpha=0.4)
london_pubs.plot(ax=ax, linewidth=0.1, facecolor='#F9DA95', alpha=1.0, edgecolor='#AE4B16', markersize=2)
# Display
plt.tight_layout()
plt.show()    

## Saving maps to figures 

Once we have produced a map we are content with, we might want to save it to a file so we can include it into a report, article, website, etc. Exporting maps in Python is as simple as replacing `plt.show` by `plt.savefig` at the end of the code block to specify where and how to save it. For example to save the previous map into a `png` file in the same folder where the notebook is hosted:

In [ ]:
# Setup figure and axis
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')
ax.set_title('Greater London MSOAs+tube+pubs')

# Plot layer of polygons on the axis
msoas_london.plot(ax=ax, linewidth=0.05, facecolor='#D5E3D8', edgecolor='#D5E3D8', alpha=0.8)
london_tube.plot(ax=ax, linewidth=0.6, edgecolor='#618A98', alpha=0.4)
london_pubs.plot(ax=ax, linewidth=0.1, facecolor='#F9DA95', alpha=1.0, edgecolor='#AE4B16', markersize=2)
# Save to file
plt.savefig('figs/london_tube_pubs.png', dpi=500, transparent=True, tight_layout=True)

# Choropleths

## Unique values

A choropleth for categorical variables simply assigns a different color to every potential value in the series. The main requirement in this case is then for the color scheme to reflect the fact that different values are not ordered or follow a particular scale.

In Python, thanks to `geopandas`, creating categorical choropleths is possible with one line of code. To demonstrate this, we can plot the spatial distribution of MSOAs with a more white population than other races and viceversa:

In [ ]:
msoas_london['majority_race'] = np.where(msoas_london.eval("white_p > 50"), "White", "Other")

In [ ]:
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

ax.axis('off')

msoas_london.plot(
    ax=ax,
    column='majority_race',
    categorical=True,
    alpha=1, 
    edgecolor='w', 
    linewidth=0.1, 
    legend=True, 
    antialiased=True
)

plt.tight_layout()
plt.show()

Let us stop for a second in a few crucial aspects:

* Note how we are using the same approach as for basic maps, the command `plot`, but we now need to add the argument `column` to specify which column in particular is to be represented.
* Since the variable is categorical we need to make that explicit by setting the argument `categorical` to `True`.
* As an optional argument, we can set `legend` to `True` and the resulting figure will include a legend with the names of all the values in the map.
* Unless we specify a different colormap, the selected one respects the categorical nature of the data by not implying a gradient or scale but a qualitative structure.

## Equal intervals

If, instead of categorical variables, we want to display the geographical distribution of a continuous phenomenon, we need to select a way to encode each value into a color. One potential solution is applying what is usually called "equal intervals". The intuition of this method is to split the *range* of the distribution, the difference between the minimum and maximum value, into equally large segments and to assign a different color to each of them according to a palette that reflects the fact that values are ordered.

Using the example of the position of a MSOA in the national ranking of the IMD (`imd_rank`), we can calculate these segments, also called bins or buckets, using the library `PySAL`:

In [ ]:
imd_attribute = 'crime'

In [ ]:
classes = ps.Equal_Interval(msoas_london[imd_attribute], k=7)
classes

The only additional argument to pass to `Equal_Interval`, other than the actual variable we would like to classify is the number of segments we want to create, `k`, which we are arbitrarily setting to seven in this case. This will be the number of colors that will be plotted on the map so, although having several can give more detail, at some point the marginal value of an additional one is fairly limited, given the ability of the brain to tell any differences.

Once we have classified the variable, we can check the actual break points where values stop being in one class and become part of the next one:

In [ ]:
classes.bins

In [ ]:
def draw_bins(geodf, attribute, scheme):

    schemes = {'equal_interval': ps.Equal_Interval, \
               'quantiles': ps.Quantiles, \
               'fisher_jenks': ps.Fisher_Jenks}
    
    classi = schemes[scheme](geodf[attribute], k=7)
    
    # Set up the figure
    f, ax = plt.subplots(1)
    # Plot the kernel density estimation (KDE)
    sns.kdeplot(geodf[attribute], shade=True)
    # Add a blue tick for every value at the bottom of the plot (rugs)
    sns.rugplot(geodf[attribute], alpha=0.5)
    # Loop over each break point and plot a vertical red line
    for cut in classi.bins:
        plt.axvline(cut, color='red', linewidth=0.75)
    # Display image
    plt.show()

draw_bins(msoas_london, imd_attribute, 'equal_interval')

Technically speaking, the figure is created by overlaying a KDE plot with vertical bars for each of the break points. This makes much more explicit the issue highlighed by which the first bin contains a large amount of observations while the one with top values only encompasses a handful of them.

To create a map that displays the colors assigned by the equal interval classification algorithm, we use a similar approach as with unique values but with some key differences:

In [ ]:
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

ax.axis('off')
ax.set_title("London IMD 2015")

msoas_london.plot(
    ax=ax,
    column=imd_attribute, 
    scheme='equal_interval', k=7, 
    cmap=plt.cm.Blues, 
    alpha=1, 
    edgecolor='w', 
    linewidth=0.1, 
    legend=True, 
    antialiased=True
)

plt.tight_layout()
plt.show()

We can use the parameter `legend_kwds` to customize the legend appearance (make sure you install geopandas directly from the github repository otherwise the `legend_kwds` parameter will not work properly as of now):

In [ ]:
f, ax = plt.subplots(
    1, 
    figsize=(12., 8.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

ax.axis('off')
ax.set_title("London IMD 2015")

msoas_london.plot(
    ax=ax,
    column=imd_attribute, 
    scheme='equal_interval', k=7, 
    cmap=plt.cm.Blues, 
    alpha=1, 
    edgecolor='w', 
    linewidth=0.1, 
    legend=True
)

# Display data sources
smallprint = ax.text(
    .995, 0,
    'Classification Method: Equal Interval\n \
    Contains National Statistics data\n \
    Contains Ordnance Survey data on %d MSOAs\n \
    $\copyright$ eurosys 2018' % int(msoas_london[imd_attribute].count()),
    ha='right', va='bottom',
    size=10,
    color='#555555',
    transform=ax.transAxes)

plt.tight_layout()
plt.show()


The custom class CustomGeoDataFrame has been introduced to solve an inconsistency (i.e., the impossibility of use the parameter legend_kwds) in the current geopandas pip package implementation. The issue is likely to be solved in the next realeses, however, in this tutorial we will make use - when necessary - of the modified version. 

In [ ]:
from geopandas import GeoDataFrame
from geopandas.plotting import __pysal_choro
from geopandas.plotting import plot_polygon_collection
from geopandas.plotting import plot_linestring_collection
from geopandas.plotting import plot_point_collection
from geopandas.plotting import plot_series

class CustomGeoDataFrame(GeoDataFrame):
    
        def plot(self, *args, **kwargs):
            """Generate a plot of the geometries in the ``GeoDataFrame``.

            If the ``column`` parameter is given, colors plot according to values
            in that column, otherwise calls ``GeoSeries.plot()`` on the
            ``geometry`` column.

            Wraps the ``plot_dataframe()`` function, and documentation is copied
            from there.
            """
            
            return plot_dataframe(self, *args, **kwargs)

In [ ]:
def plot_dataframe(df, column=None, cmap=None, color=None, ax=None,
                   categorical=False, legend=False, scheme=None, k=5, breaks=None, 
                   vmin=None, vmax=None, markersize=None, figsize=None,
                   legend_kwds=None, **style_kwds):
   
        
    if 'colormap' in style_kwds:
        warnings.warn("'colormap' is deprecated, please use 'cmap' instead "
                      "(for consistency with matplotlib)", FutureWarning)
        cmap = style_kwds.pop('colormap')
    if 'axes' in style_kwds:
        warnings.warn("'axes' is deprecated, please use 'ax' instead "
                      "(for consistency with pandas)", FutureWarning)
        ax = style_kwds.pop('axes')
    if column and color:
        warnings.warn("Only specify one of 'column' or 'color'. Using "
                      "'color'.", UserWarning)
        column = None

    import matplotlib
    import matplotlib.pyplot as plt

    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
        ax.set_aspect('equal')

    if df.empty:
        warnings.warn("The GeoDataFrame you are attempting to plot is "
                      "empty. Nothing has been displayed.", UserWarning)
        return ax

    if isinstance(markersize, str):
        markersize = df[markersize].values

    if column is None:
        return plot_series(df.geometry, cmap=cmap, color=color, ax=ax,
                           figsize=figsize, markersize=markersize,
                           **style_kwds)

    if df[column].dtype is np.dtype('O'):
        categorical = True

    # Define `values` as a Series
    if categorical:
        if cmap is None:
            if LooseVersion(matplotlib.__version__) >= '2.0.1':
                cmap = 'tab10'
            elif LooseVersion(matplotlib.__version__) >= '2.0.0':
                # Erroneous name.
                cmap = 'Vega10'
            else:
                cmap = 'Set1'
        categories = list(set(df[column].values))
        categories.sort()
        valuemap = dict([(k, v) for (v, k) in enumerate(categories)])
        values = np.array([valuemap[k] for k in df[column]])
    else:
        values = df[column]
    
    
    if scheme is not None:
    
        binning = __pysal_choro(values, scheme, k=k)
        # set categorical to True for creating the legend
        categorical = True
        binedges = [values.min()] + binning.bins.tolist()
        categories = ['{0:.2f} - {1:.2f}'.format(binedges[i], binedges[i+1])
                      for i in range(len(binedges)-1)]
        values = np.array(binning.yb)

    mn = values.min() if vmin is None else vmin
    mx = values.max() if vmax is None else vmax
    
    geom_types = df.geometry.type
    poly_idx = np.asarray((geom_types == 'Polygon')
                          | (geom_types == 'MultiPolygon'))
    line_idx = np.asarray((geom_types == 'LineString')
                          | (geom_types == 'MultiLineString'))
    point_idx = np.asarray((geom_types == 'Point')
                          | (geom_types == 'MultiPoint'))

    # plot all Polygons and all MultiPolygon components in the same collection
    polys = df.geometry[poly_idx]
    if not polys.empty:
        plot_polygon_collection(ax, polys, values[poly_idx],
                                vmin=mn, vmax=mx, cmap=cmap, **style_kwds)

    # plot all LineStrings and MultiLineString components in same collection
    lines = df.geometry[line_idx]
    if not lines.empty:
        plot_linestring_collection(ax, lines, values[line_idx],
                                   vmin=mn, vmax=mx, cmap=cmap, **style_kwds)

    # plot all Points in the same collection
    points = df.geometry[point_idx]
    if not points.empty:
        if isinstance(markersize, np.ndarray):
            markersize = markersize[point_idx]
        plot_point_collection(ax, points, values[point_idx], vmin=mn, vmax=mx,
                              markersize=markersize, cmap=cmap,
                              **style_kwds)

    if legend and not color:
        from matplotlib.lines import Line2D
        from matplotlib.colors import Normalize
        from matplotlib import cm

        norm = Normalize(vmin=mn, vmax=mx)
        n_cmap = cm.ScalarMappable(norm=norm, cmap=cmap)
        if categorical:
            patches = []
            for value, cat in enumerate(categories):
                patches.append(
                    Line2D([0], [0], linestyle="none", marker="o",
                           alpha=style_kwds.get('alpha', 1), markersize=10,
                           markerfacecolor=n_cmap.to_rgba(value)))
            if legend_kwds is None:
                legend_kwds = {}
            legend_kwds.setdefault('numpoints', 1)
            legend_kwds.setdefault('loc', 'best')
            ax.legend(patches, categories, **legend_kwds)
        else:
            n_cmap.set_array([])
            ax.get_figure().colorbar(n_cmap, ax=ax)

    plt.draw()
    return ax

In [ ]:
msoas_london = CustomGeoDataFrame(msoas_london)

In [ ]:
legend_kwds_default = {
    'loc':'lower right', 
    'fontsize':10, 
    'shadow':True, 
    'fancybox':True, 
    'frameon':True,
    'framealpha':1.0,
    'edgecolor':'#999999',
    'facecolor':'#FFFFFF',
    'bbox_to_anchor':(1, 0.1)
}

def draw_choropleth(geodf, attribute, scheme, cmap=plt.cm.viridis_r, dpi=200, width=12., 
                    height=8., title=None,legend_kwds=legend_kwds_default, smallprint=None):
    f, ax = plt.subplots(
    1, 
    figsize=(width, height),
    dpi=dpi,
    subplot_kw=dict(aspect='equal'))

    ax.axis('off')
    
    if title:
        ax.set_title(title)

    geodf.plot(
        ax=ax,
        column=attribute, 
        scheme=scheme, 
        k=7, 
        cmap=cmap, 
        alpha=1, 
        edgecolor='w', 
        linewidth=0.1, 
        legend=True, 
        antialiased=True,
        legend_kwds=legend_kwds
    )

    # Display data sources
    if smallprint:
        ax.text(smallprint,
            ha='right', va='bottom',
            size=10,
            color='#555555',
            transform=ax.transAxes)

    plt.tight_layout()
    plt.show()
    
    
    

## Quantiles 

One solution to obtain a more balanced classification scheme is using quantiles. This, by definition, assigns the same amount of values to each bin: the entire series is laid out in order and break points are assigned in a way that leaves exactly the same amount of observations between each of them. This "observation-based" approach contrasts with the "value-based" method of equal intervals and, although it can obscure the magnitude of extreme values, it can be more informative in cases with skewed distributions.

Calculating a quantiles classification with `PySAL` can be done with the following line of code:

In [ ]:
classes = ps.Quantiles(msoas_london[imd_attribute], k=7)
classes

And, similarly, the bins can also be inspected:

In [ ]:
classes.bins

The visualization of the distribution can be generated in a similar way as well:

In [ ]:
draw_bins(msoas_london, imd_attribute, 'quantiles')

And the choropleth also follows a similar pattern, with the difference that we are now using the scheme "quantiles", instead of "equal interval":

In [ ]:
draw_choropleth(msoas_london, imd_attribute, 'quantiles', title="London IMD 2015")

## Fisher-Jenks

Equal interval and quantiles are only two examples of very many classification schemes to encode values into colors. Although not all of them are integrated into `geopandas`, `PySAL` includes several other classification schemes (for a detailed list, have a look at this [link](http://pysal.readthedocs.org/en/latest/library/esda/mapclassify.html)). As an example of a more sophisticated one, let us create a Fisher-Jenks choropleth:

In [ ]:
classes = ps.Fisher_Jenks(msoas_london[imd_attribute], k=7)
classes

In [ ]:
classes.bins

In [ ]:
draw_bins(msoas_london, imd_attribute, 'fisher_jenks')

Technically, however, the way to create a Fisher-Jenks map is exactly the same as before (it takes time!):

In [ ]:
draw_choropleth(msoas_london, imd_attribute, 'fisher_jenks', title="London IMD 2015")

In [ ]:
def plot_scheme(scheme, var, db, figsize=(16, 6), saveto=None):
    '''
    Plot the distribution over value and geographical space of variable `var` using scheme `scheme
    ...
    
    Arguments
    ---------
    scheme   : str
               Name of the classification scheme to use 
    var      : str
               Variable name 
    db       : GeoDataFrame
               Table with input data
    figsize  : Tuple
               [Optional. Default = (16, 8)] Size of the figure to be created.
    saveto   : None/str
               [Optional. Default = None] Path for file to save the plot.
    '''
    schemes = {'equal_interval': ps.Equal_Interval, \
               'quantiles': ps.Quantiles, \
               'fisher_jenks': ps.Fisher_Jenks}
    classi = schemes[scheme](db[var], k=7)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, dpi=300)
    # KDE
    sns.kdeplot(db[var], shade=True, color='#26828E', ax=ax1)
    sns.rugplot(db[var], alpha=0.5, color='#26828E', ax=ax1)
    for cut in classi.bins:
        ax1.axvline(cut, color='#31688E', linewidth=0.75)
    ax1.set_title('Value distribution')
    # Map
    p = db.plot(column=var, scheme=scheme, alpha=0.75, k=7, \
             cmap='viridis_r', ax=ax2, linewidth=0.1)
    ax2.axis('equal')
    ax2.set_axis_off()
    ax2.set_title('Geographical distribution')
    f.suptitle(scheme, size=25)
    if saveto:
        plt.savefig(saveto)
    plt.show()

In [ ]:
plot_scheme('equal_interval', imd_attribute, msoas_london)

## Zooming into the map (trivial)

A general map of an entire region, or urban area, can sometimes obscure particularly local patterns because they happen at a much smaller scale that cannot be perceived in the global view. One way to solve this is by providing a focus of a smaller part of the map in a separate figure. Although there are many ways to do this in Python, the most straightforward one is to reset the limits of the axes to center them in the area of interest.

As an example, let us consider the quantile map produced above:

In [ ]:
f, ax = plt.subplots(
    1, 
    figsize=(4., 3.),
    dpi=200,
    subplot_kw=dict(aspect='equal'))

ax.axis('off')

msoas_london.plot(
    ax=ax,
    column=imd_attribute, 
    scheme='quantiles', k=7, 
    cmap=plt.cm.Blues, 
    edgecolor='black',
    lw=0.05,
    alpha=1.0
)

# Redimensionate X and Y axes to desired bounds
ax.set_ylim(175000, 185000)
ax.set_xlim(530000, 540000)
plt.tight_layout()
plt.show()


## Zooming into the map (advanced)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

boroughs_selected = ['E09000007', 'E09000033','E09000019','E09000020','E09000001']
scheme = 'Quantiles'

# Load London boroughs
boroughs_df = gpd.read_file("data/shp/London_Borough.shp")
boroughs_df = boroughs_df.loc[boroughs_df['GSS_CODE'].isin(boroughs_selected)]
boroughs_df.head()

In [ ]:
fig, ax = plt.subplots(
    1,
    figsize=(16., 12.),
    dpi=200,
    subplot_kw=dict(aspect='equal'),
)

msoas_london.loc[msoas_london['LAD11CD'].isin(boroughs_selected)].plot(
#     categorical=True,
    ax=ax,
    alpha=1.,
    column=imd_attribute,
    k=7,
    cmap="viridis_r",
    linewidth=0.1,
    edgecolor='black',
    legend=True, 
    scheme=scheme,
)

# Plot boroughs boundaries
boroughs_df.plot(ax=ax, lw=2, color="none", edgecolor="#555555")

_ = ax.axis('off')

ax.set_title("%s rates in %s London Boroughs" % (imd_attribute, len(boroughs_selected)), color='#555555')

smallprint = ax.text(
    .995, 0,
    'Classification Method: %s\n \
    Contains National Statistics data\n \
    Contains Ordnance Survey data on %d MSOAs\n \
    $\copyright$ eurosys 2017' % (scheme, msoas_london['LAD11CD'].isin(boroughs_selected).sum()),
    ha='right', va='bottom',
    size=10,
    color='#555555',
    transform=ax.transAxes)

axins = inset_axes(
    ax,
    width="20%", # width = % of parent_bbox
    height=2.0,
    loc=2,
    axes_kwargs={'aspect': 'equal', 'facecolor': '#555555'}
)

inset= msoas_london.plot(
    ax=axins,
    color='#555555',
    facecolor='#555555',
    edgecolor='#555555',
    legend=False,
)

union = boroughs_df.unary_union
union_gdf = gpd.GeoDataFrame(geometry=[union])
union_gdf.plot(ax=axins, linewidth=1, edgecolor='red', color='none')

_ = axins.axis('off')
plt.show()




## Conditional maps

Conditional maps are an attempt to explore multivariate relationships within a choropleth mapping context. In essence, they are figures composed by several choropleths in which the layout of each of them provides information about the subset of the original dataset represented. The idea is that a dataset can be subset based on one or two conditional variables, usually categorical, and only the observations that meet each characteristic are displayed in a given submap. Since they are combinations of choropleths, they build on everything we have learned about their creation. As an example, let us create a conditional map of IMD scores based on the dominating gender and marital status of each area.
From a Python perspective, creating conditional maps is a bit more intricate than simple choropleths because of the conditioning of the data and the arranging of the layout that needs to occur for the final figure to be produced. To be able to use the facetting machinery available in seaborn, we need to define a function that generates a choropleth with a given subset of the dataset:

In [ ]:
msoas_london['Status_Majority'] = np.where(msoas_london.eval("p_couple_h+p_couple_1+p_other_mu>50"), "Couple", "Single")
msoas_london['Race_Majority'] = np.where(msoas_london.eval("white_p>50"), "White", "Other")
msoas_london.head()

In [ ]:
def map_subset(vals, db, color=None, norm=True):
    '''
    Internal function to pass to `FaceGrid` to build a single map
    ...
    
    Arguments
    ---------
    vals     : Series
               Values of the subplot to be mapped
    db       : GeoDataFrame
               Table with geometries
    color    : None
    '''

    ax = plt.gca()
    db.plot(color='0.8', edgecolor='0.8', linewidth=1., ax=ax)
    vari = vals.name
    if norm:
        db.reindex(vals.index).plot(column=vari, ax=ax, cmap='viridis', linewidth=0., \
                               vmin=db[vari].min(), vmax=db[vari].max(), legend=False)
    else:
        db.reindex(vals.index).plot(column=vari, ax=ax, cmap='viridis', linewidth=0.)
    
    ax.set_axis_off()


With this function in hand, we can use it to pass it on to the facetting functionality in `seaborn`, which then takes care of the actual subsetting of the data and proper alignment of the output figures:

In [ ]:
g = sns.FacetGrid(msoas_london, row="Race_Majority", col="Status_Majority", \
                  margin_titles=True, size=4, aspect=1.2)
g.map(map_subset, imd_attribute, db=msoas_london)
plt.tight_layout()
plt.show()

The figure contains a few interesting elements:

* The distribution of areas with different characteristics is not random over space but rather follows a specific pattern. For example, the majority of married/female areas are located in the periphery of the city, while most of the single/male MSOAs can be found in the city centre. 
* Since the color scale is common across maps, we can compare the degree of deprivation for different combinations. For example, areas with a more married population display consistently lower levels of deprivation than those where singles prevail.

Although conditional maps are a powerful tool to explore a dataset and generate hypotheses about multivariate relationships, it is important to keep in mind these can only be suggestive. A more formal analysis, such as one based on regression, would be required to establish more robust conclusions, as several confounding factors can be at play.

### Exercise

Imagine some of the MSOAs do not have a value for the property crime. Try to plot the corresponding choropleth dealing with missing values visually. 

In [ ]:
msoas_london_missing = msoas_london.sample(frac=0.8)
msoas_london_missing.plot()